<a href="https://colab.research.google.com/github/dongyoung0/Kaggle-Dacon/blob/main/bluehouse/bluehouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [3]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 448 kB 53.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [4]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import csv
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tqdm import tqdm

In [5]:
def save_csv(path, data):
  with open(path, 'w',newline='', encoding='utf-8-sig') as f:   
    # using csv.writer method from CSV package 
    write = csv.writer(f) 
    #write.writerow(fields) 
    write.writerows(data)

def load_csv(path):
  load_list = []
  f =  open(path, 'r', encoding='utf-8')
  reader = csv.reader(f) 
  for row in reader:
    load_list.append(row)
  f.close()
  return load_list

In [6]:
train = pd.read_csv('./MyDrive/MyDrive/DACON/bluehouse/data/train.csv', encoding = 'utf-8')
test = pd.read_csv('./MyDrive/MyDrive/DACON/bluehouse/data/test.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('./MyDrive/MyDrive/DACON/bluehouse/data/sample_submission.csv', encoding = 'utf-8')

In [23]:
train[train['data'].isnull()]

,index,category,data
2819,2819,0,NaN
3462,3462,1,NaN
20934,20934,2,NaN
22856,22856,2,NaN
26550,26550,2,NaN
28546,28546,2,NaN
31400,31400,1,NaN
36070,36070,0,NaN


In [8]:
# 전처리

## 전처리
아래 과정을 실행한 결과를 train_okt.csv, test_okt.csv에 저장해 놨음  
load_csv로 불러다 쓰면 댐

In [9]:
train = train.dropna(how = 'any')
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
okt=Okt()

In [11]:
X_train = []
for sentence,i in zip(train['data'],tqdm(range(len(train['data'])))) :
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

100%|█████████▉| 39991/39992 [43:03<00:00, 15.48it/s]


In [19]:
X_test = []
for sentence in test['data']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

In [60]:
save_csv('./MyDrive/MyDrive/DACON/bluehouse/data/train_okt.csv', X_train)
save_csv('./MyDrive/MyDrive/DACON/bluehouse/data/test_okt.csv', X_test)

## 불러오기
위에서 okt로 전처리한 train, test 데이터 불러오기

In [31]:
X_train = load_csv('./MyDrive/MyDrive/DACON/bluehouse/data/train_okt.csv')
X_test = load_csv('./MyDrive/MyDrive/DACON/bluehouse/data/test_okt.csv')

In [32]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [33]:
vocab_size = 30000
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [34]:
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [35]:
# 결측치는 일단 제외하고 훈련
y_train = to_categorical(train['category'][train['data'].isnull()==False])

In [36]:
# 모델 생성 및 훈련

In [37]:
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
313/313 [==============================] - 40s 108ms/step - loss: 0.7793 - acc: 0.6741
Epoch 2/15
313/313 [==============================] - 33s 106ms/step - loss: 0.4146 - acc: 0.8451
Epoch 3/15
313/313 [==============================] - 33s 107ms/step - loss: 0.2547 - acc: 0.9076
Epoch 4/15
313/313 [==============================] - 33s 107ms/step - loss: 0.1912 - acc: 0.9318
Epoch 5/15
313/313 [==============================] - 33s 106ms/step - loss: 0.1515 - acc: 0.9454
Epoch 6/15
313/313 [==============================] - 33s 106ms/step - loss: 0.1271 - acc: 0.9538
Epoch 7/15
313/313 [==============================] - 33s 106ms/step - loss: 0.1169 - acc: 0.9564
Epoch 8/15
313/313 [==============================] - 33s 106ms/step - loss: 0.1069 - acc: 0.9614
Epoch 9/15
313/313 [==============================] - 33s 106ms/step - loss: 0.1085 - acc: 0.9604
Epoch 10/15
313/313 [==============================] - 33s 107ms/step - loss: 0.0938 - acc: 0.9675
Epoch 11/15
313/313

In [ ]:
#6. 훈련된 모델로 예측, submission 파일을 생성합니다.

In [39]:
model.p

In [41]:
#6. 훈련된 모델로 예측, submission 파일을 생성합니다.

#y_pred = model.predict_classes(X_test)
y_pred=model.predict(X_test) 

In [57]:

#sample_submission['category'] = y_pred
sample_submission['category'] = y_pred.argmax(axis=1)
sample_submission.to_csv('./MyDrive/MyDrive/DACON/bluehouse/data/sub2.csv', encoding='utf-8', index = False)

In [43]:
y_pred

array([[8.6823744e-01, 3.1087143e-04, 1.3145159e-01],
       [4.4585625e-04, 2.5446725e-05, 9.9952877e-01],
       [9.6713775e-06, 9.9998868e-01, 1.7113066e-06],
       ...,
       [4.1019652e-05, 9.9994206e-01, 1.6893631e-05],
       [9.9982339e-01, 1.3088658e-04, 4.5717396e-05],
       [9.2722642e-01, 3.6093870e-03, 6.9164202e-02]], dtype=float32)

In [45]:
model.save('./MyDrive/MyDrive/DACON/bluehouse/model1')

INFO:tensorflow:Assets written to: ./MyDrive/MyDrive/DACON/bluehouse/model1/assets


INFO:tensorflow:Assets written to: ./MyDrive/MyDrive/DACON/bluehouse/model1/assets
